In [ ]:
import requests
import pandas as pd
import re
import numpy as np

In [ ]:
def get_vancancy(number_of_pages,job_title,per_page):
  id_vac=[]
  colums=['']
  for job in job_title:
      data=[]
      for i in range(number_of_pages):
          url = 'https://api.hh.ru/vacancies'
          par = {'text': job,'per_page':str(per_page) ,'page':i}
          r = requests.get(url, params=par)
          e=r.json()
          data.append(e)

          vacancy_details = data[0]['items'][0].keys()
          vacancy_details=list(vacancy_details)+['responsibility']
          df = pd.DataFrame(columns= vacancy_details)
          ind = 0
          for i in range(len(data)):
              for j in range(len(data[i]['items'])):
                  df.loc[ind] = data[i]['items'][j]
                  ind+=1
  return df

In [ ]:
def transform_df(df):
  for i in range(df.shape[0]):
    df['area'][i]=df['area'][i]['name']
    df['type'][i]=df['type'][i]['name']
    if df['salary'][i]:
      if (df['salary'][i]['to']):
        df['salary'][i]=int(df['salary'][i]['to'])
      else:
        df['salary'][i]=int(df['salary'][i]['from'])
    df['employer'][i]=df['employer'][i]['name']
    tmp=df['snippet'][i]
    if df['snippet'][i]['requirement']:
        df['snippet'][i]=tmp['requirement']
        df['snippet'][i]=df['snippet'][i].replace('<highlighttext>','')
        df['snippet'][i]=df['snippet'][i].replace('</highlighttext>','')
    else:
        df['snippet'][i]=tmp['responsibility']
    df['responsibility'][i]=tmp['responsibility']
    if df['responsibility'][i]:
      df['responsibility'][i]=df['responsibility'][i].replace('<highlighttext>','')
      df['responsibility'][i]=df['responsibility'][i].replace('</highlighttext>','')
  return df

In [ ]:
def get_full_inform_vacancy(ids):
  df_descrip=pd.DataFrame(columns= ['id','full_descrip','full_req','full_resp','skill_list','experience'])
  for id in ids:
    r = requests.get(url_vac%id)
    r=r.json()
    key_skills=[]
    for j in range(len(r['key_skills'])):
        key_skills.append(r['key_skills'][j]['name'])
    df_descrip=df_descrip.append({'id':id,'full_descrip':r['description'],'skill_list':key_skills,'experience':r['experience']['name']}, ignore_index=True)
  return df_descrip




In [ ]:
url_vac = 'https://api.hh.ru/vacancies/%s'
colums_to_drop= ['premium','department','has_test','response_letter_required','address','relations','contacts','schedule','working_days','working_time_intervals','working_time_modes','accept_temporary','response_url','insider_interview','url','sort_point_distance','apply_alternate_url','published_at','created_at','archived']
tags=['<ul>','</ul>','</p>','<p>','</li>','<li>','<strong>','</strong>','<ol>','</ol>','●','<em>','</em>']



In [ ]:
df_analyt=get_vancancy(5,["'Тестировщик' and 'QA-инженер'"], 100)
df_analyt.drop(columns = colums_to_drop,axis = 1,inplace=True)
df_analyt=transform_df(df_analyt)

In [ ]:
df_analyt

,id,name,area,salary,type,adv_response_url,alternate_url,employer,snippet,responsibility
0,55455622,QA Engineer/Тестировщик,Санкт-Петербург,215000,Открытая,https://api.hh.ru/vacancies/55455622/adv_respo...,https://hh.ru/vacancy/55455622,Софтвайс,"Опыт автоматизации тестов на JS/TS, Java, etc....",Ручное тестирование Веб-приложений. Автоматиза...
1,66502987,QA Engineer / Тестировщик,Екатеринбург,100000,Открытая,https://api.hh.ru/vacancies/66502987/adv_respo...,https://hh.ru/vacancy/66502987,WARPOINT ARENA,Опыт работы от 1 года. Знание процессов тестир...,Проводить функциональное тестирование продукто...
2,66201937,QA Engineer (удалённо),Москва,4000,Открытая,https://api.hh.ru/vacancies/66201937/adv_respo...,https://hh.ru/vacancy/66201937,TextMagic,Опыт работы в тестировании от 3 лет. Опыт напи...,Функциональное тестирование (Back-end Front-e...
3,66199149,QA Engineer,Тбилиси,2300,Открытая,https://api.hh.ru/vacancies/66199149/adv_respo...,https://hh.ru/vacancy/66199149,InvestEngine,От трёх лет опыта тестирования веб- и мобильны...,Подготавливать сценарии для автоматизации. Уча...
4,66530571,Junior QA Engineer (Manual),Тбилиси,1300,Открытая,https://api.hh.ru/vacancies/66530571/adv_respo...,https://hh.ru/vacancy/66530571,Lightspeed,Analytical mind: when you can notice non-obvio...,To look for problems and bugs in the platform ...
...,...,...,...,...,...,...,...,...,...,...
495,55141720,QA engineer (Manual),Калининград,None,Открытая,https://api.hh.ru/vacancies/55141720/adv_respo...,https://hh.ru/vacancy/55141720,IBS,Опыт работы в ручном тестировании от 2х лет. З...,Планирование и проведение ручного тестирования...
496,55540739,Ручной тестировщик (Middle QA engineer),Рязань,None,Открытая,https://api.hh.ru/vacancies/55540739/adv_respo...,https://hh.ru/vacancy/55540739,"Иннотех, Группа компаний",Опыт работы на позиции инженера по качеству (Q...,Участвовать в проведении тестирования продукто...
497,54194049,Тестировщик / QA Engineer,Москва,None,Открытая,https://api.hh.ru/vacancies/54194049/adv_respo...,https://hh.ru/vacancy/54194049,Performance Lab (Перфоманс Лаб),Опыт работы в тестировании от полугода. Опыт н...,"Проведение функционального, регрессионного и и..."
498,55540738,Ручной тестировщик (Middle QA engineer),Москва,None,Открытая,https://api.hh.ru/vacancies/55540738/adv_respo...,https://hh.ru/vacancy/55540738,"Иннотех, Группа компаний",Опыт работы на позиции инженера по качеству (Q...,Участвовать в проведении тестирования продукто...


In [ ]:
df_analyt_full=get_full_inform_vacancy(list(df_analyt['id']))

In [ ]:
df_analyt_full

In [ ]:
def find_tag(tag_start,tag_end,data):
     list_index_start=[m.start() for m in re.finditer(tag_start, data)]
     list_index_end=[m.start() for m in re.finditer(tag_end, data)]
     return list_index_start,list_index_end

In [ ]:

def find_index(full_desc,short_desc,ul_start,ul_end,diff=0):
  if short_desc:
    word_index_l=[m.start() for m in re.finditer(' ', short_desc )]
    if len(word_index_l)==1:
      index=word_index_l[0]
    elif len(word_index_l)==0:
      index=len(short_desc)
    else:
      index=word_index_l[1]
    full_desc=full_desc.lower()
    short_str=short_desc[:index]
    if short_str[:3]=='...':
        short_str=short_desc[word_index_l[3]:word_index_l[5]]
    if short_str[-1] in ['.',';',':',' ']:
        short_str=short_str[:-1]
    if short_str[0]==' ':
        short_str=short_str[1:]
    print(short_str)
    start_index=full_desc.find(short_str.lower())
    print(start_index)
    ind=-1
    for i in range(len(ul_start)-diff):
      if ul_start[i]<start_index and ul_end[i+diff]>start_index:
        return i
    return -1
  return -1

In [ ]:

def find_text(list_id,df, df_descrip, df_tag,descrip_tag,diff,tag_find):
  prob_l=[]
  tag_start=f'<{tag_find}>'
  tag_end=f'</{tag_find}>'
  if diff==1 and tag_find=='strong':
    tag_start, tag_end= tag_end, tag_start
  for id in list_id:
    print(f'id:{id}')
    data=df[df.id==id]
    data_descrip=df_descrip[df_descrip.id==id]
    list_index_start,list_index_end=find_tag(tag_start,tag_end,list(data_descrip['full_descrip'])[0])
    index_strong=find_index(list(data_descrip['full_descrip'])[0],list(data[df_tag])[0],list_index_start,list_index_end,diff)
    print(index_strong)
    if index_strong!=-1:
        tmp=list(data_descrip['full_descrip'])[0][list_index_start[index_strong]:list_index_end[index_strong+diff]]
        print(tmp)
        # for tag in tags:
        #     tmp=tmp.replace(tag,'')
        df_descrip.loc[df_descrip.id==id,descrip_tag]=tmp
    else:
        prob_l.append(id)
  return df_descrip, prob_l
      

In [ ]:
list_req=[]
list_res=[]
id_transform=list(df_analyt['id'])
df_analyt_full,list_req=find_text(id_transform,df_analyt,df_analyt_full,'snippet','full_req',0,'ul')
df_analyt_full,list_res=find_text(id_transform,df_analyt,df_analyt_full,'responsibility','full_resp',0,'ul')


id:54819685
Большой игровой
752
1
<ul> <li> <p>Большой игровой опыт на мобильных платформах;</p> </li> <li> <p>Грамотная письменная и устная речь;</p> </li> <li> <p>Понимание базовых принципов тестирования;</p> </li> <li> <p>Навыки работы с баг-трекинговой системой;</p> </li> <li> <p>Знание английского на уровне B2: необходимо для оформления баг-репортов, правильного понимания писем от издателей, ведения технической переписки.</p> </li> 
id:54498013
Стек разработки
1241
2
<ul> <li>*nix системы (у нас CentOS);</li> <li>баг-трекеры (JIRA, Redmine);</li> <li>CD/CI системы (у нас TeamCity с активным использованием Gradle и Ansible скриптов);</li> <li>TCMS (у нас TestRail);</li> <li>git (у нас GitLab).</li> </ul> </li> <li>Стек разработки (<strong>большим бонусом будет</strong> <strong>опыт работы с указанными технологиями</strong> на уровне чтения-понимания кода и конфигурирования инструментов): <ul> <li>Бэкенд пишется на <strong>Java</strong> (фреймворк Spring), а тесты на <strong>Groovy<

In [ ]:
df_analyt_full

,id,full_descrip,full_req,full_resp,skill_list,experience
0,54819685,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,<ul> <li> <p>Проведение тестирования игр: тест...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта
1,54498013,<p>Привет! Мы расширяем проект и ищем специали...,<ul> <li>*nix системы (у нас CentOS);</li> <li...,<ul> <li>Непосредственное участие в проектиров...,[],От 1 года до 3 лет
2,54830968,<p>Мы IT-компания в сфере adtech и gamedev.<br...,"<ul> <li>Опыт автоматизации тестов на JS/TS, J...",<ul> <li>Ручное тестирование Веб-приложений.</...,"[YouTrack, Английский язык, PostgreSQL, Kafka,...",От 1 года до 3 лет
3,54820306,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,<ul> <li> <p>Проведение тестирования игр: тест...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта
4,54053648,<p>Команда <strong>КИБЕР-РОМ</strong> — это ИТ...,<ul> <li> <p>Понимание гибких методологий разр...,<ul> <li> <p>Участие в формировании требований...,"[Scrum, Git, REST API, Unix, SQL, Юмор]",От 3 до 6 лет
...,...,...,...,...,...,...
495,54739021,<p><strong>Привет! На связи мобильная команда ...,<ul> <li>Отличные аналитические способности.</...,"<ul> <li>Тестирование (функциональное, нагрузо...","[Тестирование пользовательского интерфейса, Ан...",От 1 года до 3 лет
496,54940188,"<p>1221Systems – компания в сфере Фудтех, спец...",<ul> <li>опыт работы в области тестирования мо...,<ul> <li>тестировать мобильные приложения (iOS...,"[Функциональное тестирование, Тестирование, Ре...",От 3 до 6 лет
497,54633563,<p><strong>Иннотех </strong>— современная быст...,<ul> <li>Опыт работы в качестве специалиста по...,<ul> <li>Проводить функциональное и регрессион...,"[Atlassian Jira, SQL, SOAP, Atlassian Confluen...",От 3 до 6 лет
498,54940187,"<p>1221Systems – компания в сфере Фудтех, спец...",<ul> <li>опыт работы в области тестирования мо...,<ul> <li>тестировать мобильные приложения (iOS...,"[Функциональное тестирование, Тестирование, Ре...",От 3 до 6 лет


In [ ]:
def to_xml_df(df,name_file):
  df.to_xml(f'{name_file}.xml')

In [ ]:
new_list_req=[]
new_list_res=[]
df_analyt_full,new_list_req=find_text(list_req,df_analyt,df_analyt_full,'snippet','full_req',1,'strong')
df_analyt_full,new_list_res=find_text(id_transform,df_analyt,df_analyt_full,'responsibility','full_resp',1,'strong')

id:55043850
3 года
-1
-1
id:54968331
Знание SQL
-1
-1
id:54914132
Самостоятельный опыт
1121
-1
id:55035555
Опыт работы
738
4
</strong> <p>- Опыт работы в качестве специалиста по интеграционному тестированию (тестирование API /Postman / Swagger );<br />- Знание процессов разработки и жизненного цикла ПО;<br />- Опыт работы с микросервисной архитектурой</p> 
id:49685851
Иметь желание
1255
3
</strong></p> <p>— Проводить ручное тестирование нового функционала, выполнять регрессионное тестирование, запускать набор авто-тестов;</p> <p>— Писать документацию;</p> <p>— Консультировать разработчиков;</p> <p>— Иметь желание ещё больше развиваться в предметной области, разбираться в функционале приложений;</p> <p> </p> <p>
id:54433521
Понимание процесса
487
0
</strong></p> <p>- Понимание процесса тестирования (цели и задачи, роли, виды тестирования, тест-дизайн, etc.)</p> <p>- Опыт функционального тестирования, тестирования web-приложений, работы с тестовой документацией и баг-трекинговыми система

In [ ]:
df_analyt_full

,id,full_descrip,full_req,full_resp,skill_list,experience
0,54819685,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,</strong></p> <ul> <li> <p>Проведение тестиров...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта
1,54498013,<p>Привет! Мы расширяем проект и ищем специали...,<ul> <li>*nix системы (у нас CentOS);</li> <li...,</strong></p> <ul> <li>Непосредственное участи...,[],От 1 года до 3 лет
2,54830968,<p>Мы IT-компания в сфере adtech и gamedev.<br...,"<ul> <li>Опыт автоматизации тестов на JS/TS, J...",</strong></p> <ul> <li>Ручное тестирование Веб...,"[YouTrack, Английский язык, PostgreSQL, Kafka,...",От 1 года до 3 лет
3,54820306,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,</strong></p> <ul> <li> <p>Проведение тестиров...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта
4,54053648,<p>Команда <strong>КИБЕР-РОМ</strong> — это ИТ...,<ul> <li> <p>Понимание гибких методологий разр...,</strong></p> <ul> <li> <p>Участие в формирова...,"[Scrum, Git, REST API, Unix, SQL, Юмор]",От 3 до 6 лет
...,...,...,...,...,...,...
495,54739021,<p><strong>Привет! На связи мобильная команда ...,<ul> <li>Отличные аналитические способности.</...,</strong></p> <ul> <li>Тестирование (функциона...,"[Тестирование пользовательского интерфейса, Ан...",От 1 года до 3 лет
496,54940188,"<p>1221Systems – компания в сфере Фудтех, спец...",<ul> <li>опыт работы в области тестирования мо...,</strong></p> <ul> <li>тестировать мобильные п...,"[Функциональное тестирование, Тестирование, Ре...",От 3 до 6 лет
497,54633563,<p><strong>Иннотех </strong>— современная быст...,<ul> <li>Опыт работы в качестве специалиста по...,</strong></p> <ul> <li>Проводить функционально...,"[Atlassian Jira, SQL, SOAP, Atlassian Confluen...",От 3 до 6 лет
498,54940187,"<p>1221Systems – компания в сфере Фудтех, спец...",<ul> <li>опыт работы в области тестирования мо...,</strong></p> <ul> <li>тестировать мобильные п...,"[Функциональное тестирование, Тестирование, Ре...",От 3 до 6 лет


In [ ]:
len(new_list_req), len(new_list_res)


(40, 47)

In [ ]:
res = df_analyt_full.merge(df_analyt, on=["id"])

In [ ]:
res

,id,full_descrip,full_req,full_resp,skill_list,experience,name,area,salary,type,alternate_url,employer,snippet,responsibility
0,54819685,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,</strong></p> <ul> <li> <p>Проведение тестиров...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта,QA Engineer / Тестировщик ПО,Санкт-Петербург,90000,Открытая,https://hh.ru/vacancy/54819685,Matryoshka Games,Большой игровой опыт на мобильных платформах. ...,Проведение тестирования игр: тестирование порт...
1,54498013,<p>Привет! Мы расширяем проект и ищем специали...,<ul> <li>*nix системы (у нас CentOS);</li> <li...,</strong></p> <ul> <li>Непосредственное участи...,[],От 1 года до 3 лет,QA Engineer,Санкт-Петербург,150000,Открытая,https://hh.ru/vacancy/54498013,Digital Security,Стек разработки (большим бонусом будет опыт ра...,"Непосредственное участие в проектировании, ана..."
2,54830968,<p>Мы IT-компания в сфере adtech и gamedev.<br...,"<ul> <li>Опыт автоматизации тестов на JS/TS, J...",</strong></p> <ul> <li>Ручное тестирование Веб...,"[YouTrack, Английский язык, PostgreSQL, Kafka,...",От 1 года до 3 лет,QA Engineer/Тестировщик,Санкт-Петербург,215000,Открытая,https://hh.ru/vacancy/54830968,Софтвайс,"Опыт автоматизации тестов на JS/TS, Java, etc....",Ручное тестирование Веб-приложений. Автоматиза...
3,54820306,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,</strong></p> <ul> <li> <p>Проведение тестиров...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта,QA Engineer / Тестировщик ПО,Новосибирск,90000,Открытая,https://hh.ru/vacancy/54820306,Matryoshka Games,Большой игровой опыт на мобильных платформах. ...,Проведение тестирования игр: тестирование порт...
4,54053648,<p>Команда <strong>КИБЕР-РОМ</strong> — это ИТ...,<ul> <li> <p>Понимание гибких методологий разр...,</strong></p> <ul> <li> <p>Участие в формирова...,"[Scrum, Git, REST API, Unix, SQL, Юмор]",От 3 до 6 лет,QA инженер / специалист по тестированию,Москва,200000,Открытая,https://hh.ru/vacancy/54053648,КИБЕР-РОМ,Понимание гибких методологий разработки (Agile...,Участие в формировании требований к продукту и...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,54739021,<p><strong>Привет! На связи мобильная команда ...,<ul> <li>Отличные аналитические способности.</...,</strong></p> <ul> <li>Тестирование (функциона...,"[Тестирование пользовательского интерфейса, Ан...",От 1 года до 3 лет,QA Engineer (Mobile),Санкт-Петербург,None,Открытая,https://hh.ru/vacancy/54739021,TradingView,Отличные аналитические способности. Наблюдател...,"Тестирование (функциональное, нагрузочное, юза..."
496,54940188,"<p>1221Systems – компания в сфере Фудтех, спец...",<ul> <li>опыт работы в области тестирования мо...,</strong></p> <ul> <li>тестировать мобильные п...,"[Функциональное тестирование, Тестирование, Ре...",От 3 до 6 лет,QA Engineer (Middle/Senior),Санкт-Петербург,None,Открытая,https://hh.ru/vacancy/54940188,1221Системс,Опыт работы в области тестирования мобильного ...,Активно участвовать в организации и развитии п...
497,54633563,<p><strong>Иннотех </strong>— современная быст...,<ul> <li>Опыт работы в качестве специалиста по...,</strong></p> <ul> <li>Проводить функционально...,"[Atlassian Jira, SQL, SOAP, Atlassian Confluen...",От 3 до 6 лет,Тестировщик / QA инженер,Москва,None,Открытая,https://hh.ru/vacancy/54633563,"Иннотех, Группа компаний",Опыт работы в качестве специалиста по функцион...,Проводить функциональное и регрессионное тести...
498,54940187,"<p>1221Systems – компания в сфере Фудтех, спец...",<ul> <li>опыт работы в области тестирования мо...,</strong></p> <ul> <li>тестировать мобильные п...,"[Функциональное тестирование, Тестирование, Ре...",От 3 до 6 лет,QA Engineer (Middle/Senior),Москва,None,Открытая,https://hh.ru/vacancy/54940187,1221Системс,Опыт работы в области тестирования мобильного ...,Активно участвовать в организации и развитии п...


In [ ]:
res = res.drop(np.where(res['experience'] == 'От 3 до 6 лет')[0])

In [ ]:
indx = pd.Index(range(0, res.shape[0], 1))
res = res.set_index(indx)

In [ ]:
res = res.drop(np.where(res['experience'] == 'Более 6 лет')[0])

In [ ]:
res

,id,full_descrip,full_req,full_resp,skill_list,experience,name,area,salary,type,alternate_url,employer,snippet,responsibility
0,54819685,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,</strong></p> <ul> <li> <p>Проведение тестиров...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта,QA Engineer / Тестировщик ПО,Санкт-Петербург,90000,Открытая,https://hh.ru/vacancy/54819685,Matryoshka Games,Большой игровой опыт на мобильных платформах. ...,Проведение тестирования игр: тестирование порт...
1,54498013,<p>Привет! Мы расширяем проект и ищем специали...,<ul> <li>*nix системы (у нас CentOS);</li> <li...,</strong></p> <ul> <li>Непосредственное участи...,[],От 1 года до 3 лет,QA Engineer,Санкт-Петербург,150000,Открытая,https://hh.ru/vacancy/54498013,Digital Security,Стек разработки (большим бонусом будет опыт ра...,"Непосредственное участие в проектировании, ана..."
2,54830968,<p>Мы IT-компания в сфере adtech и gamedev.<br...,"<ul> <li>Опыт автоматизации тестов на JS/TS, J...",</strong></p> <ul> <li>Ручное тестирование Веб...,"[YouTrack, Английский язык, PostgreSQL, Kafka,...",От 1 года до 3 лет,QA Engineer/Тестировщик,Санкт-Петербург,215000,Открытая,https://hh.ru/vacancy/54830968,Софтвайс,"Опыт автоматизации тестов на JS/TS, Java, etc....",Ручное тестирование Веб-приложений. Автоматиза...
3,54820306,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,</strong></p> <ul> <li> <p>Проведение тестиров...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта,QA Engineer / Тестировщик ПО,Новосибирск,90000,Открытая,https://hh.ru/vacancy/54820306,Matryoshka Games,Большой игровой опыт на мобильных платформах. ...,Проведение тестирования игр: тестирование порт...
4,54968331,<p>В нашу дружную команду разработчиков ищем <...,NaN,</strong></p> <p>Комплексное ручное тестирован...,"[Git, SQL, Тестирование, Функциональное тестир...",От 1 года до 3 лет,QA инженер (ручной тестировщик),Москва,60000,Открытая,https://hh.ru/vacancy/54968331,МБК,Знание SQL на хорошем уровне (обязательно). GI...,Комплексное ручное тестирование нового функцио...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,49210223,<p>Наша команда платежных технологий занимаетс...,</strong> <p>Теория тестирования:<br />- Практ...,</strong> <p>Теория тестирования:<br />- Практ...,"[Функциональное тестирование, SQL, SoapUI, SOA...",От 1 года до 3 лет,QA-engineer,Москва,None,Открытая,https://hh.ru/vacancy/49210223,Тинькофф,Теория тестирования: - Практический опыт в обл...,"Составление чек-листов, тест-плана под задачу...."
344,54625666,<p><strong>NOTAMEDIA</strong> компания по веб-...,<ul> <li>Опытом ручного тестирования web-прило...,</strong></p> <ul> <li>Тестирование веб-прилож...,"[QA, Тестирование]",От 1 года до 3 лет,Тестировщик ручной/QA инженер,Москва,None,Открытая,https://hh.ru/vacancy/54625666,NOTAMEDIA,Опытом ручного тестирования web-приложений от ...,Тестирование веб-приложений. Создание тестовой...
345,54862955,<p><strong>Мой МТС</strong> ― приложение для у...,<ul> <li> <p>Понимание основ ООП;</p> </li> <l...,</strong></p> <ul> <li> <p>Тестирование задач ...,"[Git, Java, PostgreSQL, Linux, Cucumber]",От 1 года до 3 лет,Auto QA engineer [Мой МТС],Москва,None,Открытая,https://hh.ru/vacancy/54862955,МТС,Понимание основ ООП. Понимание REST/SOAP. Опыт...,Тестирование задач команды BE (wsrest). Разраб...
346,49092396,<p><strong>Задачи:</strong></p> <ul> <li>Разра...,<ul> <li>Законченное высшее техническое образо...,"</strong></p> <ul> <li>Разработка креативных, ...","[Atlassian Jira, Scrum, Jmeter, Управление про...",От 1 года до 3 лет,Senior QA Engineer,Нур-Султан,None,Открытая,https://hh.ru/vacancy/49092396,НОВАРДИС КОНСАЛТИНГ,Законченное высшее техническое образование в о...,"Разработка креативных, инновационных приложени..."


In [ ]:
indx = pd.Index(range(0, res.shape[0], 1))
res = res.set_index(indx)

In [ ]:
res['full_resp'][3]

'</strong></p> <ul> <li> <p>Проведение тестирования игр: тестирование портирования игры, исследовательский тест, регрессионные проверки, стресс-тест, проверки UI, юзабилити тест, тестирование клиент-серверного взаимодействия;</p> </li> <li> <p>Релиз-менеджмент;</p> </li> <li> <p>Оформление баг-репортов в баг-трекинговой системе Jira;</p> </li> <li> <p>Разработка, актуализация и оформление тестовой документации: чек-листы, тест-планы, процессная документация QA.</p> </li> </ul> <p>'

In [ ]:
def make_sentense(ids,data,tag_s,tag_e,column):
  tags=['<li>','</li>','<p>','</p>','<strong>','</strong>', '✔','•','-']
  empty=[]
  st=f'({tag_s}.+?{tag_e})'
  print(st)
  for id in ids:
    print(id)
    if (type(data[column][id])!=np.float):
        print(id)
        if len([m.start() for m in re.finditer('<br />', data[column][id])])<2 :
          tmp=re.findall(st, data[column][id])
          if len(tmp)>0:
            print(tmp)
            for tag in tags:
              for j in range(len(tmp)):
                tmp[j]=tmp[j].replace(tag,'')
            data[column][id]=tmp
          else:
              empty.append(id)
        else:
          empty.append(id)
        
  return data ,empty

In [ ]:
res,empty_li=make_sentense(list(range(0, res.shape[0], 1)),res,'<li>','</li>','full_resp')
res,empty_p=make_sentense(empty_li,res,'<p>','</p>','full_resp')

(<li>.+?</li>)
0
0
['<li> <p>Проведение тестирования игр: тестирование портирования игры, исследовательский тест, регрессионные проверки, стресс-тест, проверки UI, юзабилити тест, тестирование клиент-серверного взаимодействия;</p> </li>', '<li> <p>Релиз-менеджмент;</p> </li>', '<li> <p>Оформление баг-репортов в баг-трекинговой системе Jira;</p> </li>', '<li> <p>Разработка, актуализация и оформление тестовой документации: чек-листы, тест-планы, процессная документация QA.</p> </li>']
1
1
['<li>Непосредственное участие в проектировании, анализе и разработке технических решений вместе с командой;</li>', '<li>Поддержание высокого качества продукта, участие в различных мероприятиях по обеспечению качества;</li>', '<li>Актуализация старых и написание новых тестовых сценариев и автоматизированных тестов;</li>', '<li>Актуализация внутренней тестовой и проектной документации.</li>']
2
2
['<li>Ручное тестирование Веб-приложений.</li>', '<li>Автоматизация тестирования (UI/API).</li>', '<li>Ведени

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


94
94
['<li> <p>Полный цикл тестирования игрового продукта</p> </li>', '<li> <p>Тестирование игровых механик, визуальных и звуковых эффектов, игрового процесса, аналитики, A/B тестов</p> </li>', '<li> <p>Мануальное тестирование, тестирование API, составление баг-репортов</p> </li>', '<li> <p>Создание и поддержка тестовой документации</p> </li>', '<li> <p>Взаимодействие с разработчиками, гейм-дизайнерами и другими членами команды для создания продукта высокого качества</p> </li>']
95
95
['<li>Тестирование ИТ-продуктов компании, в том числе платформы SaaS-сервисов, микросервисов, Web-сайтов для клиентов компании (в основном, ресторанные бренды: &quot;Якитория&quot;, &quot;Менза&quot;, &quot;Ян Примус&quot;).</li>', '<li>Записывать и анализировать пользовательские истории перед началом разработки.</li>', '<li>Писать тест-кейсы, находить ошибки в логике работы сервисов и в интерфейсе.</li>', '<li>Тестировать вручную и автоматически сервисы компании (web, API).</li>', '<li>Регрессивное тест

In [ ]:
column='full_resp'
tags_br=['<li>','</li>','<p>','</p>','<strong>','</strong>', '✔','•','-','<br />','<em>','</em>']
tmp=[]
empty_br=[]
big_l=[]
for id in empty_p:
    br=[m.start() for m in re.finditer('<br />', res[column][id])]
    tmp=[]
    for i in range(len(br)-1):
      tmp1=res[column][id][br[i]:br[i+1]]
      for tag in tags_br:
        tmp1=tmp1.replace(tag,'')
      tmp.append(tmp1)
    if len(tmp)>0:
      res[column][id]=tmp
    else:
      empty_br.append(id)

big_l

[]

In [ ]:
res = res.drop(index=empty_br)

In [ ]:
indx = pd.Index(range(0, res.shape[0], 1))
res = res.set_index(indx)

In [ ]:
res

,id,full_descrip,full_req,full_resp,skill_list,experience,name,area,salary,type,alternate_url,employer,snippet,responsibility
0,54819685,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,[ Проведение тестирования игр: тестирование по...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта,QA Engineer / Тестировщик ПО,Санкт-Петербург,90000,Открытая,https://hh.ru/vacancy/54819685,Matryoshka Games,Большой игровой опыт на мобильных платформах. ...,Проведение тестирования игр: тестирование порт...
1,54498013,<p>Привет! Мы расширяем проект и ищем специали...,<ul> <li>*nix системы (у нас CentOS);</li> <li...,"[Непосредственное участие в проектировании, ан...",[],От 1 года до 3 лет,QA Engineer,Санкт-Петербург,150000,Открытая,https://hh.ru/vacancy/54498013,Digital Security,Стек разработки (большим бонусом будет опыт ра...,"Непосредственное участие в проектировании, ана..."
2,54830968,<p>Мы IT-компания в сфере adtech и gamedev.<br...,"<ul> <li>Опыт автоматизации тестов на JS/TS, J...","[Ручное тестирование Вебприложений., Автоматиз...","[YouTrack, Английский язык, PostgreSQL, Kafka,...",От 1 года до 3 лет,QA Engineer/Тестировщик,Санкт-Петербург,215000,Открытая,https://hh.ru/vacancy/54830968,Софтвайс,"Опыт автоматизации тестов на JS/TS, Java, etc....",Ручное тестирование Веб-приложений. Автоматиза...
3,54820306,<p><strong>Приглашаем к сотрудничеству талантл...,<ul> <li> <p>Большой игровой опыт на мобильных...,[ Проведение тестирования игр: тестирование по...,"[Тестирование, Тестирование игр, GameDev, Конт...",Нет опыта,QA Engineer / Тестировщик ПО,Новосибирск,90000,Открытая,https://hh.ru/vacancy/54820306,Matryoshka Games,Большой игровой опыт на мобильных платформах. ...,Проведение тестирования игр: тестирование порт...
4,54968331,<p>В нашу дружную команду разработчиков ищем <...,NaN,[Комплексное ручное тестирование нового функци...,"[Git, SQL, Тестирование, Функциональное тестир...",От 1 года до 3 лет,QA инженер (ручной тестировщик),Москва,60000,Открытая,https://hh.ru/vacancy/54968331,МБК,Знание SQL на хорошем уровне (обязательно). GI...,Комплексное ручное тестирование нового функцио...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,49210223,<p>Наша команда платежных технологий занимаетс...,</strong> <p>Теория тестирования:<br />- Практ...,"[ Практический опыт в области тестирования;, ...","[Функциональное тестирование, SQL, SoapUI, SOA...",От 1 года до 3 лет,QA-engineer,Москва,None,Открытая,https://hh.ru/vacancy/49210223,Тинькофф,Теория тестирования: - Практический опыт в обл...,"Составление чек-листов, тест-плана под задачу...."
336,54625666,<p><strong>NOTAMEDIA</strong> компания по веб-...,<ul> <li>Опытом ручного тестирования web-прило...,"[Тестирование вебприложений;, Создание тестово...","[QA, Тестирование]",От 1 года до 3 лет,Тестировщик ручной/QA инженер,Москва,None,Открытая,https://hh.ru/vacancy/54625666,NOTAMEDIA,Опытом ручного тестирования web-приложений от ...,Тестирование веб-приложений. Создание тестовой...
337,54862955,<p><strong>Мой МТС</strong> ― приложение для у...,<ul> <li> <p>Понимание основ ООП;</p> </li> <l...,"[ Тестирование задач команды BE (ws|rest); , ...","[Git, Java, PostgreSQL, Linux, Cucumber]",От 1 года до 3 лет,Auto QA engineer [Мой МТС],Москва,None,Открытая,https://hh.ru/vacancy/54862955,МТС,Понимание основ ООП. Понимание REST/SOAP. Опыт...,Тестирование задач команды BE (wsrest). Разраб...
338,49092396,<p><strong>Задачи:</strong></p> <ul> <li>Разра...,<ul> <li>Законченное высшее техническое образо...,"[Разработка креативных, инновационных приложен...","[Atlassian Jira, Scrum, Jmeter, Управление про...",От 1 года до 3 лет,Senior QA Engineer,Нур-Султан,None,Открытая,https://hh.ru/vacancy/49092396,НОВАРДИС КОНСАЛТИНГ,Законченное высшее техническое образование в о...,"Разработка креативных, инновационных приложени..."


In [ ]:
to_xml_df(res,'full_analyt')
to_xml_df(df_analyt,'short_analyt')

In [ ]:
res=res.drop(index=[one_sen[-1]])
indx = pd.Index(range(0, res.shape[0], 1))
res = res.set_index(indx)



In [ ]:
one_sen=[]
for i in range(res.shape[0]):
  if type(res['full_resp'][i])!=np.float:
    if len(res['full_resp'][i])==1:
      print(res['full_resp'][i])
      one_sen.append(i)

['Мы разрабатываем полностью новое приложение, которое реализовывает часть функционала изначальной большой системы. В основные задачи QA входит проведение функционального тестирования, составление тестовой документации, работа с базами данных.']
['Написание тестов по составленным тесткейсам. Поддержка и развитие процесса автоматизированного тестирования.']
['']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
for id in one_sen:
   print(res['full_descrip'][id])

<p><strong>Qulix - </strong>международная IT-компания с офисами в РБ, Польше, Узбекистане и Великобритании.</p> <p>В команду департамента <strong>R&amp;D and Innovations </strong>мы ищем <strong>QA Engineer</strong> с опытом от 1 года.Наши заказчики в основном из Америки и Великобритании. Мы ищем в команду людей, которые открыты ко всему новому, хотят развиваться и любят разнообразие.</p> <p>Один из возможных проектов – из сферы <strong>Healthcare, Social Care, Medical Care.</strong></p> <p>Мы разрабатываем полностью новое приложение, которое реализовывает часть функционала изначальной большой системы. В основные задачи QA входит проведение функционального тестирования, составление тестовой документации, работа с базами данных.</p> <p><strong>Наши ожидания от вас:</strong></p> <ul> <li>практические навыки тестирования, коммерческий опыт от года;</li> <li>опыт в разработке тест-кейсов и чек-листов;</li> <li>понимание клиент-серверной архитектуры;</li> <li>опыт работы с базами данных и A

In [ ]:
for j in range(len(one_sen)):
   tmp=res['full_resp'][one_sen[j]][0][:-1].split('.')
   if i==0 or i==2:
     tmp=tmp[1:]
     tmp1=[]
     for i in range(len(tmp)):
      tmp1.append(tmp[i].split(','))
     tmp=tmp1[0]
   res['full_descrip'][one_sen[j]]=tmp
   print(tmp)


['Мы разрабатываем полностью новое приложение, которое реализовывает часть функционала изначальной большой системы', ' В основные задачи QA входит проведение функционального тестирования, составление тестовой документации, работа с базами данных']
['Написание тестов по составленным тесткейсам', ' Поддержка и развитие процесса автоматизированного тестирования']
['']


In [ ]:
res.to_csv('res.csv') 